# Becoming a Backpropagation Ninja

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print((max(len(w) for w in words)))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build vocabulary of characters and stoi, itos mappings
chars = sorted(list(set(''.join(words)))) # a to z
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(stoi)
print(itos)
print('vocab size: ', vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
vocab size:  27


In [217]:
# build the dataset
block_size = 3
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr   = build_dataset(words[:n1])     # 80%
Xdev, Ydev  = build_dataset(words[n1:n2])   # 10%
Xte,  Yte   = build_dataset(words[n2:])     # 10%

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


Boilerplate Done 

In [56]:
# utility fn to compare manual gradients with pytorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff:{maxdiff}')

In [218]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # number of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),               generator=g)                                                ;print('C\t\t', C.shape, '\n')

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),   generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)       ;print('W1\t\t', W1.shape)
# useless cause of BatchNorm layer that subtracts mean undoing the bias, but keeping it for practice
b1 = torch.randn(n_hidden,                          generator=g) * 0.1                                          ;print('b1\t\t', b1.shape, '\n')

# BatchNorm paramaters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0                                                                 ;print('bngain\t', bngain.shape)
bnbias = torch.randn((1, n_hidden)) * 0.1                                                                       ;print('bnbias\t', bnbias.shape, '\n')

# Layer 2
W2 = torch.randn((n_hidden, vocab_size),            generator=g) * 0.1                                          ;print('W2\t\t', W2.shape)
b2 = torch.randn(vocab_size,                        generator=g) * 0.1                                          ;print('b2\t\t', b2.shape, '\n')

# Note, above params are initialized in non-standard ways to ensure that we don't accidentally mask incorrect manual gradient eval (eg: all zeros scenario)
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True
    
print('params\t', sum(p.nelement() for p in parameters))

C		 torch.Size([27, 10]) 

W1		 torch.Size([30, 64])
b1		 torch.Size([64]) 

bngain	 torch.Size([1, 64])
bnbias	 torch.Size([1, 64]) 

W2		 torch.Size([64, 27])
b2		 torch.Size([27]) 

params	 4137


In [219]:
batch_size = 32
n = batch_size

# construct minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)     ;print('ix\t\t', ix.shape, '\n')

Xb, Yb = Xtr[ix], Ytr[ix]                                           ;print('Xb\t\t', Xb.shape)  ;print('Yb\t\t', Yb.shape)

ix		 torch.Size([32]) 

Xb		 torch.Size([32, 3])
Yb		 torch.Size([32])


forward pass, chunked into smaller steps, on which we can run backward pass one at a time

# Bessel's Correction

Dividing by n-1 gives better estimate where we have small populations, which is the case for mini batches. It's aka unbiased variance estimation.
If we just estimated by 1/n, then it'd almost always underestimate the variance.

[Relevant Paper](https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqblZsQ1lCeTg1S19aVkpEckNfaDhWaEt6RGpDd3xBQ3Jtc0ttbUNPc1dhOW9DQ1REYS1YMnNOQnVTaFVzazRyOEhrY2dJcVRTOVdUSHFQVDl2ZkhWX2lUQzRPV0JodTY5NjV0YXVVWm9kYzQwRnFtbkZQSS03M1BESXp1RFZ0TU1CRWdDaTZJU3o3c0FIdVk3SkZENA&q=http%3A%2F%2Fmath.oxford.emory.edu%2Fsite%2Fmath117%2FbesselCorrection%2F&v=q8SA3rM6ckI)

In [220]:
emb = C[Xb]                                                                             ;print('emb\t\t\t\t', emb.shape)
embcat = emb.view(emb.shape[0], -1)                                                     ;print('embcat\t\t\t', embcat.shape, '\n')

##
# Forward Pass
##

# Linear layer 1
hprebn = embcat @ W1 + b1                                                               ;print('hprebn\t\t\t', hprebn.shape, '\n')

# BatchNorm layer
#mean
bnmeani = (1/n)*hprebn.sum(0, keepdim=True)                                             ;print('bnmeani\t\t\t', bnmeani.shape)
#var == average of the squared differences
bndiff = hprebn - bnmeani                                                               ;print('bndiff\t\t\t', bndiff.shape)
bndiff2 = bndiff**2                                                                     ;print('bndiff2\t\t\t', bndiff2.shape)
#bassel's correction to divide by n-1, not n
bnvar = ((1/(n-1))*bndiff2).sum(0, keepdim=True)                                        ;print('bnvar\t\t\t', bnvar.shape)
bnvar_inv = (bnvar + 1e-5)**-0.5                                                        ;print('bnvar_inv\t\t', bnvar_inv.shape)
bnraw = bndiff * bnvar_inv                                                              ;print('bnraw\t\t\t', bnraw.shape)
hpreact = bngain * bnraw + bnbias                                                       ;print('hpreact\t\t\t', hpreact.shape, '\n')

# Non-linearity
h = torch.tanh(hpreact)                                                                 ;print('h\t\t\t\t', h.shape, '\n')

# Linear layer 2
logits = h @ W2 + b2                                                                    ;print('logits\t\t\t', logits.shape, '\n')

# Cross entropy loss ( same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values                                        ;print('logit_maxes\t\t', logit_maxes.shape)
#subtract max for numberical stability. If we do not do this, logits with too large values can end up causing issues.
norm_logits = logits - logit_maxes                                                      ;print('norm_logits\t\t', norm_logits.shape)
counts = norm_logits.exp()                                                              ;print('counts\t\t\t', counts.shape)
counts_sum = counts.sum(1, keepdim=True)                                                ;print('counts_sum\t\t', counts_sum.shape)
#using 1.0/counts_sum makes it hard to get backprop to be exact... something with pytorch _shrug_
counts_sum_inv = counts_sum**-1                                                         ;print('counts_sum_inv\t', counts_sum_inv.shape)
#broadcasting being done to do element wise multiplication
probs = counts * counts_sum_inv                                                         ;print('probs\t\t\t', probs.shape)
logprobs = probs.log()                                                                  ;print('logprobs\t\t', logprobs.shape)
loss = -logprobs[range(n), Yb].mean()                                                   ;print('loss\t\t\t', loss.shape)

##
# Backward Pass
##
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()

loss.backward()
loss

emb				 torch.Size([32, 3, 10])
embcat			 torch.Size([32, 30]) 

hprebn			 torch.Size([32, 64]) 

bnmeani			 torch.Size([1, 64])
bndiff			 torch.Size([32, 64])
bndiff2			 torch.Size([32, 64])
bnvar			 torch.Size([1, 64])
bnvar_inv		 torch.Size([1, 64])
bnraw			 torch.Size([32, 64])
hpreact			 torch.Size([32, 64]) 

h				 torch.Size([32, 64]) 

logits			 torch.Size([32, 27]) 

logit_maxes		 torch.Size([32, 1])
norm_logits		 torch.Size([32, 27])
counts			 torch.Size([32, 27])
counts_sum		 torch.Size([32, 1])
counts_sum_inv	 torch.Size([32, 1])
probs			 torch.Size([32, 27])
logprobs		 torch.Size([32, 27])
loss			 torch.Size([])


tensor(3.5444, grad_fn=<NegBackward0>)

In [221]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros(logprobs.size())
dlogprobs[range(n), Yb] = -1.0/n

dprobs = (1.0 / probs) * dlogprobs

dcounts_sum_inv = counts * dprobs
# if a node is used multiple times, the gradients end up summing during backpropagation, broadcasting is equivalent to using multiple times
dcounts_sum_inv = dcounts_sum_inv.sum(1, keepdims=True)

dcounts_sum = (-1 / (counts_sum**2)) * dcounts_sum_inv

# dcounts is used in two nodes, the gradients end up summing
dcounts = ((dprobs * counts_sum_inv)
            + (torch.ones_like(counts) * dcounts_sum)) 

dnorm_logits = norm_logits.exp() * dcounts

dlogit_maxes = (-dnorm_logits).sum(1, keepdims=True)

dlogits = dnorm_logits * 1
dlogits += dlogit_maxes * ((logits == logit_maxes).int())

# important derivation: https://youtu.be/q8SA3rM6ckI?feature=shared&t=2853
# d = a @ b + c
# d(L) / d(a) = (d(L) / d(d)) @ transpose(b)
dh = (dlogits @ W2.T)

# similarly, d(L) / d(b) = transpose(a) @ (d(L) / d(d)) 
dW2 = h.T @ dlogits 

# d(L) / d(c) = (d(L) / d(d)).sum(0)
db2 = dlogits.sum(0)

dhpreact = dh * (1 - torch.tanh(hpreact)**2)

dbngain = (bnraw * dhpreact).sum(0, keepdims=True)

dbnraw = bngain * dhpreact

dbnbias = dhpreact.sum(0, keepdims=True)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdims=True)

dbndiff = bnvar_inv * dbnraw

dbnvar = dbnvar_inv * (-0.5*(bnvar+1e-5)**-1.5)

dbndiff2 = torch.ones(32, 1) @ (dbnvar * (1/(n-1)))

dbndiff += dbndiff2 * 2 * bndiff

dbnmeani = (dbndiff * -1).sum(0, keepdims=True)

dhprebn = dbndiff.clone() # for safety, copies the data
dhprebn += (1.0/n) * (torch.ones_like(hprebn) * dbnmeani)

dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)

# We know demb (32x3x10), so, the only thing to do is to see how each element in C contributes to this, accounting for summing up repeated embeddings
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        dC[Xb[k,j]] += demb[k,j]
        
# print(C[Xb].shape, C.shape, Xb.shape)
# print(Xb[:5])

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff:0.0
probs           | exact: True  | approximate: True  | maxdiff:0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff:0.0
counts_sum      | exact: True  | approximate: True  | maxdiff:0.0
counts          | exact: True  | approximate: True  | maxdiff:0.0
norm_logits     | exact: True  | approximate: True  | maxdiff:0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff:0.0
logits          | exact: True  | approximate: True  | maxdiff:0.0
h               | exact: True  | approximate: True  | maxdiff:0.0
W2              | exact: True  | approximate: True  | maxdiff:0.0
b2              | exact: True  | approximate: True  | maxdiff:0.0
hpreact         | exact: True  | approximate: True  | maxdiff:0.0
bngain          | exact: True  | approximate: True  | maxdiff:0.0
bnraw           | exact: True  | approximate: True  | maxdiff:0.0
bnbias          | exact: True  | approximate: True  | maxdiff:0.0
bnvar_inv 

In [222]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.5444116592407227 diff: 4.76837158203125e-07


In [223]:
# backward pass

# -----------------
# YOUR CODE HERE :)

"""
logits
# softmax, to give probabilities from logits
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
# loss == -nll == negative log likelihood
loss = -prob[torch.arange(32), Yb].log().mean()
"""

dlogits = F.softmax(logits, 1) # columns are output for each row
dlogits[range(n), Yb] -= 1
dlogits /= n # loss scaled down by n, gradient should also be scaled down by n

# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff:6.28642737865448e-09


In [224]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [225]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = (bngain * bnvar_inv / n) * (
         (-n/(n-1))*bnraw*((bnraw*dhpreact).sum(0))
         + n*dhpreact
         - dhpreact.sum(0)
)

# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff:1.3969838619232178e-09


In [242]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad(): # basically, saying pytorch that we are not gonna call loss.backward on anything here

    # kick off optimization
    for i in range(max_steps):
    
      # minibatch construct
      ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
      Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
    
      # forward pass
      emb = C[Xb] # embed the characters into vectors
      embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
      # Linear layer
      hprebn = embcat @ W1 + b1 # hidden layer pre-activation
      # BatchNorm layer
      # -------------------------------------------------------------
      bnmean = hprebn.mean(0, keepdim=True)
      bnvar = hprebn.var(0, keepdim=True, unbiased=True)
      bnvar_inv = (bnvar + 1e-5)**-0.5
      bnraw = (hprebn - bnmean) * bnvar_inv
      hpreact = bngain * bnraw + bnbias
      # -------------------------------------------------------------
      # Non-linearity
      h = torch.tanh(hpreact) # hidden layer
      logits = h @ W2 + b2 # output layer
      loss = F.cross_entropy(logits, Yb) # loss function
    
      # backward pass
      for p in parameters:
        p.grad = None
      # loss.backward() # use this for correctness comparisons, delete it later!
    
      # manual backprop! #swole_doge_meme
      # -----------------
      dlogits = F.softmax(logits, 1) # columns are output for each row
      dlogits[range(n), Yb] -= 1
      dlogits /= n # loss scaled down by n, gradient should also be scaled down by n
      # 2nd layer backprob
      dh = (dlogits @ W2.T) 
      dW2 = h.T @ dlogits 
      db2 = dlogits.sum(0)
      # tanh backprop
      dhpreact = dh * (1 - torch.tanh(hpreact)**2)
      # batchnorm backprop
      dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
      dbnbias = dhpreact.sum(0, keepdims=True)
      dhprebn = (bngain * bnvar_inv / n) * (
              (-n/(n-1))*bnraw*((bnraw*dhpreact).sum(0))
              + n*dhpreact
              - dhpreact.sum(0)
      )
      # 1st layer
      dembcat = dhprebn @ W1.T
      dW1 = embcat.T @ dhprebn
      db1 = dhprebn.sum(0)
      # embedding
      demb = dembcat.view(emb.shape)
      dC = torch.zeros_like(C)
      for k in range(Xb.shape[0]):
          for j in range(Xb.shape[1]):
              dC[Xb[k,j]] += demb[k,j]
      
      
      # dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
      grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
      # -----------------
    
      # update
      lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
      for p, grad in zip(parameters, grads):
        # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
        p.data += -lr * grad # new way of swole doge TODO: enable
    
      # track stats
      if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
      lossi.append(loss.log10().item())
    
      # if i >= 100: # TODO: delete early breaking when you're ready to train the full net
      #   break

12297
      0/ 200000: 3.7316
  10000/ 200000: 1.7984
  20000/ 200000: 1.8494
  30000/ 200000: 2.4125
  40000/ 200000: 2.4160
  50000/ 200000: 1.8749
  60000/ 200000: 2.4931
  70000/ 200000: 2.0872
  80000/ 200000: 1.8220
  90000/ 200000: 2.3179
 100000/ 200000: 2.2750
 110000/ 200000: 2.1868
 120000/ 200000: 1.9906
 130000/ 200000: 1.8613
 140000/ 200000: 2.1606
 150000/ 200000: 2.2070
 160000/ 200000: 1.8613
 170000/ 200000: 2.2749
 180000/ 200000: 2.3869
 190000/ 200000: 1.8296


In [243]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

TypeError: all(): argument 'input' (position 1) must be Tensor, not bool

In [244]:
# calibrate the batch norm at the end of training

with torch.no_grad(): # since we aren't maintaining a running mean & variance
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [245]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0669243335723877
val 2.1246795654296875


In [250]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmahzati.
harlikimri.
reyty.
skanden.
jazonte.
delynn.
jareei.
nellara.
chaiivon.
leigh.
ham.
joce.
quintesslee.
alianni.
wavero.
dearinix.
kaellinsley.
dae.
iia.
gian.
